In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess


In [4]:
main=pd.read_csv('../data/train.csv')
sub=pd.read_csv('../data/store.csv')
data=pd.merge(main,sub,on='Store',how='left')
data['Date']=pd.to_datetime(data['Date'])
data.sort_values(by=['Store','Date'],inplace=True)
data.set_index('Date',inplace=True)
data.head()

/tmp/ipykernel_38841/1517388885.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  main=pd.read_csv('../data/train.csv')


,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Date,,,,,,,,,,,,,,,,,
2013-01-01,1,2,0,0,0,0,a,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2013-01-02,1,3,5530,668,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2013-01-03,1,4,4327,578,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2013-01-04,1,5,4486,619,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2013-01-05,1,6,4997,635,1,0,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [6]:
fouriers = CalendarFourier(freq="A", order=10)#生成日历傅里叶特征
dp = DeterministicProcess(
    index=data.index,
    constant=True,               #是否包含常数项
    order=1,                     #线性趋势
    seasonal=False,               #是否包含季节性虚拟变量
    additional_terms=[fouriers], #添加傅里叶特征
    drop=True,                   #删除重复项
)
X = dp.in_sample()              #生成特征矩阵
y = data["Sales"]               #目标变量



/home/yjjjc/python/myv1/lib/python3.12/site-packages/statsmodels/tsa/deterministic.py:569: FutureWarning: 'A' is deprecated and will be removed in a future version, please use 'YE' instead.
  index = pd.date_range("2020-01-01", freq=freq, periods=1)


In [ ]:
def make_lag(data,lags,week):
    for lag in range(1,lags+1):
            data['lag_{}_{}'.format(lag,week)]=data['Sales'].shift(lag+7*week)
    return data
temp=pd.DataFrame()
temp['Sales']=data['Sales'].copy()
make_lag(temp,3,0)
make_lag(temp,3,1)
temp.dropna(inplace=True)
temp['rolling_mean_7']=temp['Sales'].shift(1).rolling(
    windows=7,
    min_periods=1,
    center=False

).mean()

,Sales,lag_1_0,lag_2_0,lag_3_0,lag_1_1,lag_2_1,lag_3_1
Date,,,,,,,
2013-01-11,4881,4892.0,5471.0,5580.0,4327.0,5530.0,0.0
2013-01-12,4952,4881.0,4892.0,5471.0,4486.0,4327.0,5530.0
2013-01-13,0,4952.0,4881.0,4892.0,4997.0,4486.0,4327.0
2013-01-14,4717,0.0,4952.0,4881.0,0.0,4997.0,4486.0
2013-01-15,3900,4717.0,0.0,4952.0,7176.0,0.0,4997.0
